# Portfolio LONG ONLY

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import random

In [117]:
data= pd.read_csv("../data/return.csv")
sp= pd.read_csv("../data/sp500.csv")
portfolio= pd.read_csv("../data/portfolio_constitution.csv")

In [118]:
sp["return"]= 100 * (sp["Close"].shift(-1) - sp["Close"])/sp["Close"]
sp["return"]= sp["return"].shift(1)
sp_return= sp[["Date", "return"]].dropna()
sp_return.head()

,Date,return
1,2010-01-01,-3.697425
2,2010-02-01,2.851369
3,2010-03-01,5.879643
4,2010-04-01,1.475923
5,2010-05-01,-8.197584


In [119]:
# transforme le df avec les titres des portefeuilles
portfolio["portfolio"]= portfolio["portfolio"].str.strip('][').str.split(', ')
portfolio["portfolio"]= portfolio["portfolio"].apply(
    lambda row: [x.replace("'", "") for x in row])

portfolio.head()
portfolio["portfolio"][0]

['HES US Equity', 'MO US Equity', 'WELL US Equity']

In [140]:
# calcul cov et var
tickers= [i for i in data.columns if i not in ["Date1", "Date2"]]
data_tickers= data[tickers]

data_corr= data_tickers.corr()
data_cov= data_tickers.cov()


# std= np.std(data_tickers, axis= 0)
# idn= std.index

# std_dict= {idn[i]:std[i] for i in range(len(std))}
# std_dict

In [121]:
test= np.array([[23, 21, 17, 10, 9], [6, 5, 9, 7, 3], [0, -2, -6, 2, 0]])
test
w_test= np.array([[random.random() for _ in range(test.shape[1])] for _ in range(test.shape[0])])
w_test2= w_test/np.sum(w_test, axis= 0)
w_test2

array([[0.48239741, 0.44289938, 0.13456413, 0.39474659, 0.32489141],
       [0.34847612, 0.04285116, 0.58749696, 0.48478186, 0.38491984],
       [0.16912647, 0.51424946, 0.2779389 , 0.12047154, 0.29018875]])

In [122]:
# w_test2.T * le return sur toutes les périodes
w_test2.T[0]

array([0.48239741, 0.34847612, 0.16912647])

In [141]:
# ajout matrice var cov dans le dataset par portefeuille

portfolio["cov"]= portfolio["portfolio"].apply(lambda row: np.matrix(data_cov.loc[row, row]))
# save
portfolio.to_csv("../data/result_portfolio.csv", index= False, header= True)

In [142]:
data_cov.loc[portfolio["portfolio"][1], portfolio["portfolio"][1]]

,TGT US Equity,DUK US Equity,DTE US Equity,HSY US Equity,WEC US Equity
TGT US Equity,43.578263,-2.287598,-0.485471,1.547108,1.067696
DUK US Equity,-2.287598,15.092307,12.000453,6.921832,13.998862
DTE US Equity,-0.485471,12.000453,14.839588,8.117057,13.855017
HSY US Equity,1.547108,6.921832,8.117057,24.238535,8.163991
WEC US Equity,1.067696,13.998862,13.855017,8.163991,18.955652


In [161]:
w_test= np.array([[random.random() for _ in range(test.shape[1])] for _ in range(test.shape[0])])
w_test2= w_test/np.sum(w_test, axis= 0)
w_test2

array([[0.22808952, 0.18697521, 0.40384409, 0.17868834, 0.67934852],
       [0.32499413, 0.17301556, 0.17819886, 0.29064429, 0.11768561],
       [0.44691635, 0.64000922, 0.41795705, 0.53066736, 0.20296588]])

In [225]:
def poids_opti(row0, row1):
    stock_var_portfolio= {}
    for i in range(1000):
    #     random.seed(55)
#         cov_test= portfolio["cov"][1]
        cov_test= row1
        test= np.array([random.random() for _ in range(len(row0))])
        test= test/np.sum(test)
        res_var= np.asarray(np.dot(np.dot(test.T, cov_test), test))[0][0]
        stock_var_portfolio[res_var]= test
    return min(stock_var_portfolio.items(), key=lambda x: x[0])

In [226]:
## stock_var_portfolio
ok= poids_opti(portfolio["portfolio"][1], portfolio["cov"][1])
ok
# test= np.array([random.random() for _ in range(len(row0))])
# test= test/np.sum(test)
# res_var= np.asarray(np.dot(np.dot(test.T, cov_test), test))[0][0]

(9.175058601216028,
 array([0.22521891, 0.29288254, 0.25056949, 0.21998217, 0.0113469 ]))

In [222]:
portfolio["poids_var"]= portfolio.apply(lambda row: poids_opti(row[0], row[1])
                                       , axis= 1)

ValueError: ('shapes (5,) and (3,3) not aligned: 5 (dim 0) != 3 (dim 0)', 'occurred at index 1')

In [223]:
portfolio.head()

,portfolio,cov
0,"[HES US Equity, MO US Equity, WELL US Equity]",[[[[[100.41903311 6.72056771 4.09955939]]]...
1,"[TGT US Equity, DUK US Equity, DTE US Equity, ...",[[[[[43.5782631 -2.28759833 -0.48547121 1.54...
2,"[PPL US Equity, MO US Equity, DTE US Equity]","[[[[[19.51266423 6.70958138 9.18255513]]]], ..."
3,"[KMB US Equity, PPL US Equity, WEC US Equity]","[[[[[18.98891434 6.79175117 8.30634734]]]], ..."
4,"[HES US Equity, MO US Equity, DTE US Equity, W...",[[[[[100.41903311 6.72056771 4.50376371 ...
